In [ ]:
%load_ext autoreload
%autoreload 2

from collections import defaultdict

import json
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.keypoint_utils.body_parts import BodyParts
from research.utils.image_utils import Picture
from research.weight_estimation.keypoint_utils.akpr import get_homography_and_matches, generate_sift_adjustment

pd.set_option('display.max_rows', 500)

In [ ]:
body_parts = BodyParts().get_core_body_parts()
s3_access_utils = S3AccessUtils('/root/data')
modified_anns = []

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id=5
    AND captured_at BETWEEN '2019-06-05' AND '2019-07-02'
    AND keypoints is not null
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    AND is_qa = FALSE;
"""

df = rds_access_utils.extract_from_database(query)

In [ ]:
body_parts = BodyParts().get_core_body_parts()
s3_access_utils = S3AccessUtils('/root/data')
modified_anns = []

rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id=5
    AND captured_at BETWEEN '2019-06-05' AND '2019-07-02'
    AND keypoints is not null
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    AND is_qa = TRUE;
"""

df = rds_access_utils.extract_from_database(query)

In [ ]:
modified_anns = []
count = 0
analysis_data = defaultdict(list)
for idx, row in df.iterrows():
    print(count)
    count += 1
    # get annotation information
    ann = row.keypoints
    left_kps_frame = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    right_kps_frame = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    left_kps = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_kps = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}

    # get image information and metadata
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata

    modified_ann = generate_sift_adjusted_ann(body_parts, left_crop_metadata, left_crop_url, left_kps,
                                              right_crop_metadata, right_crop_url, right_kps, s3_access_utils)
    

    
        
    


In [ ]:
count = 0
keypoint_data, analysis_data = defaultdict(list), defaultdict(list)
adj_anns = []
for idx, row in df.head(1000).iterrows():
    print(count)
    count += 1
    
    # get image information and metadata
    left_crop_url, right_crop_url = row.left_image_url, row.right_image_url
    left_crop_metadata, right_crop_metadata = row.left_crop_metadata, row.right_crop_metadata
    
    # get keypoint coordinates
    ann = row.keypoints
    left_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in ann['leftCrop']}
    right_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in ann['rightCrop']}
    left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in ann['leftCrop']}
    right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in ann['rightCrop']}
    
    # jitter keypoints
    jitter_magnitude = 15
    jittered_left_kps = {bp: left_kps[bp] + np.array([int(np.random.normal(0, jitter_magnitude)), 0]) for bp in left_kps.keys()}
    jittered_right_kps = {bp: right_kps[bp] + np.array([int(np.random.normal(0, jitter_magnitude)), 0]) for bp in right_kps.keys()}
    jittered_left_kps_frame = {bp: jittered_left_kps[bp] + np.array([left_crop_metadata['x_coord'], left_crop_metadata['y_coord']]) 
                               for bp in body_parts}
    jittered_right_kps_frame = {bp: jittered_right_kps[bp] + np.array([right_crop_metadata['x_coord'], right_crop_metadata['y_coord']]) 
                               for bp in body_parts}
    

    left_fish_picture = Picture(s3_access_utils=s3_access_utils, image_url=left_crop_url)
    right_fish_picture = Picture(s3_access_utils=s3_access_utils, image_url=right_crop_url)
    left_fish_picture.enhance(in_place=True)
    right_fish_picture.enhance(in_place=True)
    sift = cv2.KAZE_create()
    left_items, right_items = [], []
    for bp in body_parts:
        
        try:
            left_item, right_item, num_matches = generate_sift_adjustment(bp, left_crop_metadata, left_fish_picture,
                                                                          jittered_left_kps, right_crop_metadata,
                                                                          right_fish_picture, jittered_right_kps, sift)
        except:
            continue
        left_items.append(left_item)
        right_items.append(right_item)
        
        original_disp = abs(left_kps_frame[bp][0] - right_kps_frame[bp][0])
        jittered_disp = abs(jittered_left_kps_frame[bp][0] - jittered_right_kps_frame[bp][0])
        adj_disp = abs(left_item['xFrame'] - right_item['xFrame'])
        
        analysis_data['kpid'].append(row.id)
        analysis_data['body_part'].append(bp)
        analysis_data['original_disp'].append(original_disp)
        analysis_data['jittered_disp'].append(jittered_disp)
        analysis_data['adj_disp'].append(adj_disp)
        analysis_data['num_matches'].append(num_matches)
    
    adj_ann = {
        'leftCrop': left_items,
        'rightCrop': right_items
    }
    keypoint_data['kpid'].append(row.id)
    keypoint_data['left_crop_url'].append(left_crop_url)
    keypoint_data['right_crop_url'].append(right_crop_url)
    keypoint_data['ann'].append(ann)
    keypoint_data['adj_ann'].append(adj_ann)
    
    
analysis_df = pd.DataFrame(analysis_data)
keypoint_df = pd.DataFrame(keypoint_data)

    

In [ ]:
kpid = 241971
row = keypoint_df[keypoint_df.kpid == kpid].iloc[0]
left_crop_url, right_crop_url, ann, adj_ann = row.left_crop_url, row.right_crop_url, row.ann, row.adj_ann
left_picture, right_picture = Picture(s3_access_utils=s3_access_utils, image_url=left_crop_url), \
                              Picture(s3_access_utils=s3_access_utils, image_url=right_crop_url)
left_crop_image, right_crop_image = left_picture.get_image(), right_picture.get_image()
left_draw, right_draw = ImageDraw.Draw(left_crop_image), ImageDraw.Draw(right_crop_image)

r = 5
for item in ann['leftCrop']:
    x, y = item['xCrop'], item['yCrop']
    left_draw.ellipse((x - r, y - r, x + r, y + r), fill='red', outline='red')
for item in ann['rightCrop']:
    x, y = item['xCrop'], item['yCrop']
    right_draw.ellipse((x - r, y - r, x + r, y + r), fill='red', outline='red')

for item in adj_ann['leftCrop']:
    x, y = item['xCrop'], item['yCrop']
    left_draw.ellipse((x - r, y - r, x + r, y + r), fill='green', outline='green')
for item in adj_ann['rightCrop']:
    x, y = item['xCrop'], item['yCrop']
    right_draw.ellipse((x - r, y - r, x + r, y + r), fill='green', outline='green')

    
left_crop_image


In [ ]:
right_crop_image

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(20, 20))
for idx, bp in enumerate(body_parts):
    
    row = idx // 3
    col = idx % 3
    
    bp_mask = analysis_df.body_part == bp
    behavior_mask = (analysis_df.num_matches > 20) & (abs(analysis_df.original_disp - analysis_df.adj_disp) < 100)
    
    ax = axes[row, col]
    ax.hist(analysis_df[bp_mask].jittered_disp - analysis_df[bp_mask].original_disp, bins=50, color='red', alpha=0.6)
    ax.hist(analysis_df[bp_mask & behavior_mask].adj_disp - analysis_df[bp_mask & behavior_mask].original_disp, bins=50, color='blue', alpha=0.6)
    ax.grid()
    ax.set_title('Body part: {}'.format(bp))
plt.show()